In [42]:
import timm
import torch
import torch.nn as nn
import torchvision.models as models


from torchinfo import summary
from thop import profile
from pprint import pprint

In [43]:
device = "cuda" if torch.cuda.is_available() else "cpu"
input = torch.randn(32, 2, 260, 346).to(device)

In [44]:
"""查看指定模型名字"""

model_names = timm.list_models('*mobilene*t*')
pprint(model_names)

['mobilenetv2_035',
 'mobilenetv2_050',
 'mobilenetv2_075',
 'mobilenetv2_100',
 'mobilenetv2_110d',
 'mobilenetv2_120d',
 'mobilenetv2_140',
 'mobilenetv3_large_075',
 'mobilenetv3_large_100',
 'mobilenetv3_rw',
 'mobilenetv3_small_050',
 'mobilenetv3_small_075',
 'mobilenetv3_small_100',
 'tf_mobilenetv3_large_075',
 'tf_mobilenetv3_large_100',
 'tf_mobilenetv3_large_minimal_100',
 'tf_mobilenetv3_small_075',
 'tf_mobilenetv3_small_100',
 'tf_mobilenetv3_small_minimal_100']


In [45]:
"""查看模型下载网址及默认配置"""

model_name = timm.create_model('resnet18')  # pretrained=True,
pprint(model_name.default_cfg)  # 查看模型cfg，包含输入尺寸，url等信息

{'architecture': 'mobilenetv3_large_100',
 'classifier': 'classifier',
 'crop_mode': 'center',
 'crop_pct': 0.875,
 'custom_load': False,
 'first_conv': 'conv_stem',
 'fixed_input_size': False,
 'hf_hub_id': 'timm/mobilenetv3_large_100.ra_in1k',
 'input_size': (3, 224, 224),
 'interpolation': 'bicubic',
 'mean': (0.485, 0.456, 0.406),
 'num_classes': 1000,
 'pool_size': (7, 7),
 'std': (0.229, 0.224, 0.225),
 'tag': 'ra_in1k',
 'url': 'https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv3_large_100_ra-f55367f5.pth'}


In [46]:
"""手动指定模型路径并创建模型"""

model_path = (
    "/mnt/data2T/junyuan/eye-tracking/TimmModels/mobilenetv3_large_100_ra-f55367f5.pth"
)
model = timm.create_model(
    'mobilenetv3_large_100',  # 模型名字
    pretrained=False,  # 是否加载预训练权重
    num_classes=2,  # 调整最后一层的分类个数
    pretrained_cfg_overlay=dict(file=model_path),  # 模型预训练权重.pth地址
    in_chans=2,  # 输入通道数
)
model = model.to(device)

In [47]:
"""查看模型输出"""

output = model(input)
print(f"Output shape: {output.shape}")
features = model.forward_features(input)
print(f"Features shape: {features.shape}")

Output shape: torch.Size([32, 2])
Features shape: torch.Size([32, 960, 9, 11])


In [48]:
"""查看模型结构"""

print(model)

MobileNetV3(
  (conv_stem): Conv2d(2, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): Hardswish()
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn1): BatchNormAct2d(
          16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (se): Identity()
        (conv_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
        (drop_path): Identity()
      )
    )
    (1): Sequential(
      (0): InvertedResidual(
 

In [49]:
"""查看模型结构(细致)"""

model_info = summary(model, input_size=(1, 2, 260, 346))
model_info

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV3                                   [1, 2]                    --
├─Conv2d: 1-1                                 [1, 16, 130, 173]         288
├─BatchNormAct2d: 1-2                         [1, 16, 130, 173]         32
│    └─Identity: 2-1                          [1, 16, 130, 173]         --
│    └─Hardswish: 2-2                         [1, 16, 130, 173]         --
├─Sequential: 1-3                             [1, 960, 9, 11]           --
│    └─Sequential: 2-3                        [1, 16, 130, 173]         --
│    │    └─DepthwiseSeparableConv: 3-1       [1, 16, 130, 173]         464
│    └─Sequential: 2-4                        [1, 24, 65, 87]           --
│    │    └─InvertedResidual: 3-2             [1, 24, 65, 87]           3,440
│    │    └─InvertedResidual: 3-3             [1, 24, 65, 87]           4,440
│    └─Sequential: 2-5                        [1, 40, 33, 44]           --
│    │    └─

In [50]:
"""查看模型参数量"""

flops, params = profile(model, inputs=(input,))  # 计算量，参数量
print(flops, params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
12907397376.0 4177488.0


In [51]:
"""查看/设定模型的图像增强方法"""

timm.data.resolve_data_config(model.pretrained_cfg)

{'input_size': (3, 224, 224),
 'interpolation': 'bicubic',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'crop_pct': 0.875,
 'crop_mode': 'center'}

In [52]:
model.classifier

Linear(in_features=1280, out_features=2, bias=True)

In [53]:
class CustomMobileNetV3(nn.Module):
    def __init__(self, num_classes=6, in_channels=2):
        super(CustomMobileNetV3, self).__init__()
        self.model = timm.create_model(
            'mobilenetv3_large_100',  # 模型名字
            pretrained=False,  # 是否加载预训练权重
            num_classes=num_classes,  # 调整最后一层的分类个数
            pretrained_cfg_overlay=dict(file=model_path),  # 模型预训练权重.pth地址
            in_chans=in_channels,  # 输入通道数
        )

        # 移除最后的分类头
        self.model.reset_classifier(0)

        # 添加自定义的检测头
        self.detection_head = nn.Sequential(
            nn.Linear(self.model.num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)  # 展平
        x = self.detection_head(x)
        return x

In [56]:
"""查看自定义模型输出"""

custom_model = CustomMobileNetV3(num_classes=6, in_channels=2).to(device)
output = custom_model(input)
print(f"Intput shape: {input.shape}")
print(f"Output shape: {output.shape}")
features = model.forward_features(input)
print(f"Features shape: {features.shape}")

Intput shape: torch.Size([32, 2, 260, 346])
Output shape: torch.Size([32, 6])
Features shape: torch.Size([32, 960, 9, 11])


In [ ]:
output

tensor([[ 0.0618, -0.0701, -0.0509, -0.0214,  0.0026,  0.0872],
        [ 0.0861, -0.0561, -0.0747,  0.0035, -0.0167,  0.0777],
        [ 0.1032, -0.0900, -0.0226, -0.0275,  0.0326,  0.0623],
        [ 0.0544, -0.0778, -0.0198, -0.0247, -0.0156,  0.1227],
        [ 0.0717, -0.0774, -0.0556, -0.0266, -0.0050,  0.1446],
        [ 0.0829, -0.0359, -0.0943, -0.0395, -0.0033,  0.0957],
        [ 0.0758, -0.1040, -0.0052, -0.0291,  0.0394,  0.0862],
        [ 0.0643, -0.0067,  0.0007,  0.0456, -0.0523,  0.0532],
        [ 0.0295, -0.0753,  0.0334,  0.0473, -0.0476,  0.0811],
        [ 0.0772, -0.0331, -0.0128,  0.0213, -0.0356,  0.0479],
        [ 0.0661, -0.0081, -0.0189,  0.0276,  0.0143,  0.0793],
        [ 0.0446, -0.0459, -0.0372, -0.0136,  0.0180,  0.0745],
        [ 0.0536, -0.0630, -0.0593, -0.0320, -0.0130,  0.0704],
        [ 0.0690, -0.0660, -0.0837, -0.0295, -0.0274,  0.1182],
        [ 0.0842, -0.0802, -0.0530, -0.0335,  0.0065,  0.1116],
        [ 0.0186, -0.0477, -0.0234,  0.0